In [ ]:
ad = dbutils.secrets.get('#SERVICE_PRINCIPAL_NAME#', '#TENANT_ID#')
sp = dbutils.secrets.get('#SERVICE_PRINCIPAL_NAME#', '#SP_ID#')
se = dbutils.secrets.get('#SERVICE_PRINCIPAL_NAME#', '#SECRET_KEY#')

 
spark.conf.set("fs.azure.account.auth.type.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", sp)
spark.conf.set("fs.azure.account.oauth2.client.secret.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", se)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "https://login.microsoftonline.com/" + ad + "/oauth2/token")

In [ ]:
import dlt
input_file = "abfss://sphealthcare2@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/dbo.Mkt_Campaigns.csv"
@dlt.table()
def Campaigns_bronze():
  return (spark.read.format("csv").option("header",True).load(input_file))

@dlt.table()
def Campaigns_silver():
  return (dlt.read("Campaigns_bronze"))

@dlt.table()
def Campaigns_gold():
  return (dlt.read("Campaigns_silver"))